In [1]:
import plotly.graph_objs as go
import plotly.io as pio
import xgboost as xgb
import pandas as pd
import numpy as np

from pandas import date_range, Series, concat
from flask import Flask, request, render_template
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
def create_plot(data, ylabel, platform, title, release_date):
    trace = go.Scatter(
        x=data.index,
        y=data.values,
        mode='lines+markers',
        name=ylabel,
        hovertemplate='%{x|%b %Y}<br>' + ylabel + ': %{y}<extra></extra>'
    )

    release_line = go.Scatter(
        x=[release_date, release_date],
        y=[data.min(), data.max()],
        mode='lines',
        line=dict(color='red', dash='dash'),
        name='5G Release (June 2019)'
    )

    layout = go.Layout(
        title=f"{title} ({platform})",
        xaxis=dict(title='Month'),
        yaxis=dict(title=ylabel),
        hovermode='x unified'
    )

    fig = go.Figure(data=[trace, release_line], layout=layout)
    return pio.to_html(fig, full_html=False)

In [3]:
def create_bar_chart(data, title, x_title, y_title):
    traces = []
    for col in data.columns:
        traces.append(go.Bar(
            x=data.index,
            y=data[col],
            name=col
        ))

    layout = go.Layout(
        title=title,
        xaxis=dict(title=x_title),
        yaxis=dict(title=y_title),
        barmode='group'
    )

    fig = go.Figure(data=traces, layout=layout)
    return pio.to_html(fig, full_html=False)


In [4]:
def create_pie_chart(data, title):
    trace = go.Pie(
        labels=data.index,
        values=data.values,
        name='Revenue Distribution',
        hoverinfo='label+percent',
        textinfo='value+percent'
    )

    layout = go.Layout(title=title)
    
    fig = go.Figure(data=[trace], layout=layout)
    return pio.to_html(fig, full_html=False)

In [5]:
def create_trend_graphs(df_filtered, platform):
    
    # Daily Active Users (DAU)
    dau_trend = df_filtered.groupby('Month')['Daily Active Users (DAU)'].sum()
    dau_trend.index = dau_trend.index.to_timestamp()
    dau_plot_url = create_plot(dau_trend, "Average Daily Active Users", platform, "Monthly Active Users Trend Over Time", "2019-06-01")

    # New Registrations
    new_registrations = df_filtered.groupby('Month')['New Registrations'].sum()
    new_registrations.index = new_registrations.index.to_timestamp()
    new_reg_plot_url = create_plot(new_registrations, "New Registrations", platform, "Monthly New Registrations Trend Over Time", "2019-06-01")

    # Session Duration (minutes)
    session_duration = df_filtered.groupby('Month')['Session Duration (minutes)'].mean()
    session_duration.index = session_duration.index.to_timestamp()
    session_plot_url = create_plot(session_duration, "Average Session Duration (minutes)", platform, "Monthly Average Session Duration Trend Over Time", "2019-06-01")

    # Revenue ($)
    revenue = df_filtered.groupby('Month')['Revenue ($)'].mean()
    revenue.index = revenue.index.to_timestamp()
    revenue_plot_url = create_plot(revenue, "Revenue ($)", platform, "Monthly Revenue Trend Over Time", "2019-06-01")

    return dau_plot_url, new_reg_plot_url, session_plot_url, revenue_plot_url

In [6]:
def create_network_graphs():
    network_df = pd.read_csv('network_metrics_summary.csv')

    # Convert numeric columns
    for col in ['Avg RTT (s)', 'Bandwidth (Mbps)', 'Avg Jitter (s)', 'Packet Loss (packets)']:
        network_df[col] = pd.to_numeric(network_df[col], errors='coerce')

    # Prepare pivot tables
    rtt_data = network_df.pivot_table(index='Game', columns='Connection', values='Avg RTT (s)')
    bw_data = network_df.pivot_table(index='Game', columns='Connection', values='Bandwidth (Mbps)')
    jitter_data = network_df.pivot_table(index='Game', columns='Connection', values='Avg Jitter (s)')
    loss_data = network_df.pivot_table(index='Game', columns='Connection', values='Packet Loss (packets)')

    # Create charts
    rtt_html = create_bar_chart(rtt_data, "Average RTT (s) by Game and Connection", "Game", "RTT (s)")
    bw_html = create_bar_chart(bw_data, "Bandwidth (Mbps) by Game and Connection", "Game", "Bandwidth (Mbps)")
    jitter_html = create_bar_chart(jitter_data, "Average Jitter (s) by Game and Connection", "Game", "Jitter (s)")
    loss_html = create_bar_chart(loss_data, "Packet Loss by Game and Connection", "Game", "Packets Lost")

    return rtt_html, bw_html, jitter_html, loss_html


In [7]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / np.clip(y_true, 1e-10, None))) * 100

In [8]:
def train_and_forecast_revenue(df_filtered, forecast_months):
    drop_cols = ['Platform', 'Social Media Mentions', 'Stream Viewership', 'Top Genre', 'Influencer Endorsements']
    df = df_filtered.drop(columns=drop_cols, errors='ignore')

    df['Date'] = pd.to_datetime(df['Date'])
    df['Month'] = df['Date'].dt.to_period('M')

    monthly_df = df.groupby('Month').agg({
        'Daily Active Users (DAU)': 'sum',
        'New Registrations': 'sum',
        'Session Duration (minutes)': 'mean',
        'Revenue ($)': 'mean'
    }).reset_index()

    monthly_df['Month'] = monthly_df['Month'].dt.to_timestamp()

    for lag in range(1, 4):
        monthly_df[f'Revenue_Lag_{lag}'] = monthly_df['Revenue ($)'].shift(lag)

    monthly_df.dropna(inplace=True)

    X = monthly_df[['Revenue_Lag_1', 'Revenue_Lag_2', 'Revenue_Lag_3',
                    'Daily Active Users (DAU)', 'New Registrations', 'Session Duration (minutes)']]
    y = monthly_df['Revenue ($)']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1)
    model.fit(X_train, y_train)

    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    mae = mean_absolute_error(y_test, preds)
    mape = mean_absolute_percentage_error(y_test, preds)

    future_preds = []
    last_row = monthly_df.iloc[-1:].copy()

    for _ in range(forecast_months):
        X_future = last_row[['Revenue_Lag_1', 'Revenue_Lag_2', 'Revenue_Lag_3',
                             'Daily Active Users (DAU)', 'New Registrations', 'Session Duration (minutes)']]
        next_revenue = model.predict(X_future)[0]
        future_preds.append(next_revenue)

        new_row = last_row.copy()
        new_row['Revenue_Lag_3'] = new_row['Revenue_Lag_2']
        new_row['Revenue_Lag_2'] = new_row['Revenue_Lag_1']
        new_row['Revenue_Lag_1'] = next_revenue
        new_row['Revenue ($)'] = next_revenue
        last_row = new_row

    return future_preds, rmse, mae, mape

In [9]:
def create_future_plot(df_filtered, future_revenue, forecast_months): 
    historical_revenue = df_filtered.groupby(df_filtered['Date'].dt.to_period('M'))['Revenue ($)'].mean()
    historical_revenue.index = historical_revenue.index.to_timestamp()

    last_date = historical_revenue.index.max()
    future_dates = date_range(start=last_date + pd.offsets.MonthBegin(1), periods=forecast_months, freq='MS')
    future_revenue_series = Series(future_revenue, index=future_dates)

    trace_actual = go.Scatter(
        x=historical_revenue.index,
        y=historical_revenue.values,
        mode='lines+markers',
        name="Actual Revenue",
        line=dict(color='blue'),
        hovertemplate='%{x|%b %Y}<br>Revenue ($): %{y}<extra></extra>'
    )

    trace_forecast = go.Scatter(
        x=future_revenue_series.index,
        y=future_revenue_series.values,
        mode='lines+markers',
        name="Forecasted Revenue",
        line=dict(color='green', dash='dash'),
        hovertemplate='%{x|%b %Y}<br>Revenue ($): %{y}<extra></extra>'
    )

    release_line = go.Scatter(
        x=["2019-06-01", "2019-06-01"],
        y=[min(historical_revenue.min(), future_revenue_series.min()), max(historical_revenue.max(), future_revenue_series.max())],
        mode='lines',
        line=dict(color='red', dash='dot'),
        name='5G Release (June 2019)'
    )

    layout = go.Layout(
        title="Actual vs Forecasted Monthly Revenue (All Platforms)",
        xaxis=dict(title='Month'),
        yaxis=dict(title='Revenue ($)'),
        hovermode='x unified'
    )

    fig = go.Figure(data=[trace_actual, trace_forecast, release_line], layout=layout)
    revenue_plot_url = pio.to_html(fig, full_html=False)

    return revenue_plot_url

In [ ]:
app = Flask(__name__)

df = pd.read_csv('dataset/Gaming-Trends-2024.csv')

df['Date'] = pd.to_datetime(df['Date'])

df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

df['Month'] = df['Date'].dt.to_period('M')

@app.route('/', methods=['GET', 'POST'])
def index():
    
    platform = request.form.get('platform', 'PC')
    forecast_months = int(request.form.get('forecast_months', 24))

    df_filtered = df[df['Platform'] == platform].copy()

    # Line Graph
    dau_plot_url, new_reg_plot_url, session_plot_url, revenue_plot_url = create_trend_graphs(df_filtered, platform)

    # Bar Chart
    new_reg_by_platform = df.groupby('Platform')['New Registrations'].sum()
    new_reg_bar_chart_url = create_bar_chart(new_reg_by_platform.to_frame(name='New Registrations'), "New Registrations by Platform", "Platform", "New Registrations")

    # Pie Chart
    revenue_by_genre = df.groupby('Top Genre')['Revenue ($)'].sum()
    revenue_pie_chart_url = create_pie_chart(revenue_by_genre, "Revenue Percentage by Gaming Genre")

    # Forecast Graph
    future_revenue, rmse, mae, mape = train_and_forecast_revenue(df_filtered, forecast_months)
    revenue_plot_forecast_url = create_future_plot(df_filtered, future_revenue, forecast_months)

    # Network Graph
    rtt_html, bw_html, jitter_html, loss_html = create_network_graphs()

    return render_template(
        'index.html',
        dau_plot_html=dau_plot_url,
        new_reg_plot_html=new_reg_plot_url,
        session_plot_html=session_plot_url,
        revenue_plot_html=revenue_plot_url,
        new_reg_bar_chart_html=new_reg_bar_chart_url,
        revenue_pie_chart_html=revenue_pie_chart_url,
        revenue_plot_forecast_html = revenue_plot_forecast_url,
        rmse=rmse,
        mae=mae,
        mape=mape,
        rtt_chart=rtt_html,
        bw_chart=bw_html,
        jitter_chart=jitter_html,
        loss_chart=loss_html,
        platform=platform,
        platforms=df['Platform'].unique(),
        forecast_months=forecast_months
    )

@app.route('/5g_coverage_map')
def coverage_map():
    return render_template('5g_coverage_map.html')

if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [08/Apr/2025 15:42:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2025 15:43:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2025 15:43:09] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [08/Apr/2025 15:43:29] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2025 15:44:02] "GET /5g_coverage_map HTTP/1.1" 200 -
